# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [4]:
"""print out the number of twits"""

# TODO Implement 
print(len(twits['data']))



1548010


### Split Message Body and Sentiment Score

In [5]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [6]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    #re.sub(pattern, ' ', text)
    text = re.sub(r'http\S+', ' ', text)
    
    # Replace ticker symbols with a space. The ticker symbols are 
    #any stock symbol that starts with $.
    #text = re.sub(r'\s::\w+', ' ', text)
    text = re.sub(r"[$]\w+", " ", text)
    #xt = re.sub(r"[^a-zA-Z0-9]", " ", text) 
    #r'(\s)#\w+'
    #query=re.sub(r"\bsearch\w+", "", query)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    #text = pass
    text = re.sub(r"[@]\w+", " ", text)
    # Replace everything not a letter with a space
    #text = pass
    text = re.sub('[^a-zA-Z]', " ", text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    #tokens = pass
    tokens = re.sub(r"\b[a-zA-Z]\b", "", text)
    tokens = tokens.split()
    #print(tokens)

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(w) for w in tokens]
    
    return tokens


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [7]:
# TODO Implement

#As suggested by reviewer:
tokenized = [preprocess(tweet) for tweet in messages]


### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [8]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 
tokens = []
for token in tokenized:
    for word in token:
        tokens.append(word)

#print(tokens)
bow = Counter(tokens)
#print(bow)

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [16]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""
# TODO Implement 
#
import operator

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
total_count = len(bow)
#print(total_count)
freqs = {word: count/total_count for word, count in bow.items()}
#print(len(freqs))
# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 1e-5

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 12

# The k most common words in the corpus. Use `high_cutoff` as the k.
#GITHUB solution:   https://github.com/chricke/udacity/blob/master/AITND/Sentiment_Analysis_with_Neural_Networks/project_6_starter.ipynb
K_most_common = sorted(freqs.items(), key=operator.itemgetter(1))[-high_cutoff:]
K_most_common = [x[0] for x in K_most_common]

filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
len(filtered_words)

['will', 'at', 'it', 'this', 'in', 'and', 'of', 'on', 'for', 'is', 'to', 'the']


98402

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [17]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement
from tqdm import tqdm_notebook as tqdm
# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
#From the lesson on movie sentiment
counts = Counter(filtered_words)
vocab_temp = sorted(counts, key=counts.get, reverse=True)
vocab = {word: ii for ii, word in enumerate(vocab_temp, 1)}

# Reverse of the `vocab` dictionary. The key is word id and value is the word.
id2vocab = dict(map(reversed, vocab.items()))

# tokenized with the words not in `filtered_words` removed.
filtered = [[word for word in message if word in vocab] for message in tqdm(tokenized)]



HBox(children=(IntProgress(value=0, max=1548010), HTML(value='')))

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [18]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [19]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.19509943078599976

Finally let's convert our tokens into integer ids which we can pass to the network.

In [20]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

In [21]:
print(token_ids[10])
sentiments[5]

[13, 77, 78, 79, 80, 81, 82, 83, 84, 85]


3

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [22]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [23]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement
       
        #With help from forum plus movie sentiment workbook
        # Setup embedding layer
        self.embedding = nn.Embedding(self.vocab_size, self.embed_size)
        
        # Setup additional layers
        self.lstm = nn.LSTM(self.embed_size, self.lstm_size, self.lstm_layers, 
                            dropout=self.dropout, batch_first = False)
        
        #Set Dropout
        self.dropout = nn.Dropout(self.dropout)
        
        #linear and softmax layers
        self.fc = nn.Linear(self.lstm_size, self.output_size)
        self.softmax = nn.LogSoftmax(dim = 1)
        
    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        #Modified from movie sentiment notebook
        weight = next(self.parameters()).data
        
#         if (train_on_gpu):
#             hidden_state = (weight.new(self.lstm_layers, batch_size, self.output_size).zero_().cuda(),
#                   weight.new(self.lstm_layers, batch_size, self.output_size).zero_().cuda())
#         else:
#             hidden_state = (weight.new(self.lstm_layers, batch_size, self.output_size).zero_(),
#                       weight.new(self.lstm_layers, batch_size, self.output_size).zero_())
        hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                  weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        
        return hidden

    #Modified from movie sentiment notebook
    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement 
        #batch_size = nn_input.size(0)
        batch_size = nn_input.long()
        # embeddings and lstm_out
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden_state)
    
        # stack up lstm outputs
        #lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        #from forums??????
        #Because of the output size change you shouldn't be reshaping the lstm_outs with .contiguous().view... instead you want to use lstm_out[-1, :, :] to obtain the last output of the sequence
        lstm_out = lstm_out[-1,:,:] 
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        #Softmax function
        log_probs = self.softmax(out)
        # sigmoid function
        #sig_out = self.sig(out)
        #from github???
        batch_size = nn_input.size(1)
        
        # reshape to be batch_size first
        #sig_out = sig_out.view(batch_size, -1)
        #sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return log_probs, hidden_state
        
        
        #return None, None

### View Model

In [24]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)
print(model)

tensor([[-1.9341, -1.4630, -1.2760, -1.5301, -2.0538],
        [-2.0354, -1.3370, -1.3179, -1.5094, -2.1372],
        [-2.0118, -1.3637, -1.3311, -1.4919, -2.1085],
        [-2.0104, -1.3576, -1.3050, -1.5238, -2.1226]])
TextClassifier(
  (embedding): Embedding(98402, 10)
  (lstm): LSTM(10, 6, num_layers=2, dropout=0.1)
  (dropout): Dropout(p=0.1)
  (fc): Linear(in_features=6, out_features=5, bias=True)
  (softmax): LogSoftmax()
)


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [25]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [56]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement

split_frac = 0.8

token_temp_ids = token_ids[:1000]
sentiments_temp_ids = sentiments[:1000]

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(token_ids)* split_frac)
train_features, valid_features = token_ids[:split_idx], token_ids[split_idx:]
train_labels, valid_labels = sentiments[:split_idx], sentiments[split_idx:]

# split_idx = int(len(token_temp_ids)* split_frac)
# train_features, valid_features = token_temp_ids[:split_idx], token_temp_ids[split_idx:]
# train_labels, valid_labels = sentiments_temp_ids[:split_idx], sentiments_temp_ids[split_idx:]

# test_idx = int(len(remaining_x)*0.5)
# val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
# val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Sizes:")
print("Train set: \t\t{}".format(len(train_features)), 
      "\nValidation set: \t{}".format(len(valid_features)))#,
      #"\nTest set: \t\t{}".format(test_x.shape))

# train_features = pass
# valid_features = pass
# train_labels = pass
# valid_labels = pass

			Feature Sizes:
Train set: 		825700 
Validation set: 	206425


In [57]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [58]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)
print(device)

cuda


In [ ]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""
import numpy as np
epochs = 4
batch_size = 256
learning_rate = .001

#Clipping parameter
clip = 5

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size):#, sequence_length=20, shuffle=True):
        steps += 100
        
        hidden = model.init_hidden(labels.shape[0])
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        
        for each in hidden:
            each.to(device)
        
        #hidden = tuple([each.data for each in hidden])
        # TODO Implement: Train Model
        correct = 0
        total = 1
        
        model.zero_grad()
        output, hidden = model(text_batch, hidden)
        
        loss = criterion(output.squeeze(), labels)
        loss.backward()
        #print("made it here")
        #Clip gradient
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if steps % print_every == 0:
            model.eval()
            
            # TODO Implement: Print metrics
            #TOTAL Frankenstein combination of suggestions from the student hub 
            # Get validation loss
            
            val_losses = []
            #model.eval()
            
            #with torch.no_grad():
            for inputs, labels in dataloader(valid_features,
                                             valid_labels,
                                             batch_size = batch_size): #,
                                             #value of 20 comes from above
                                             #sequence_length = 20,
                                             #shuffle = True):

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                inputs, labels = inputs.to(device), labels.to(device)
                #steps = steps + 1
                #val_h = tuple([each.data for each in val_h])
                val_h = model.init_hidden(labels.shape[0])
                for each in val_h:
                    each.to(device)
#                     if(train_on_gpu):
#                         inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = model(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels)

                val_losses.append(val_loss.item())
                
                #From pytorch docs -- https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
                _, predicted = torch.max(output.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            model.train()
            
            print("Epoch: {}/{}...".format(epoch + 1, epochs),
                  "Step: {}...".format(steps),
                  "Training Loss: {:.6f}...".format(loss.item()),
                  "Validation Loss: {:.6f}".format(np.mean(val_losses)),
                  "Accuracy: {:.6f}".format(100 * correct/total))
            #model.train()

Starting epoch 1
Epoch: 1/4... Step: 100... Training Loss: 1.281145... Validation Loss: 1.577064 Accuracy: 31.933477
Epoch: 1/4... Step: 200... Training Loss: 1.011430... Validation Loss: 1.572514 Accuracy: 29.987017
Epoch: 1/4... Step: 300... Training Loss: 1.284741... Validation Loss: 1.554693 Accuracy: 33.682773
Epoch: 1/4... Step: 400... Training Loss: 1.246030... Validation Loss: 1.603455 Accuracy: 30.137192
Epoch: 1/4... Step: 500... Training Loss: 1.422028... Validation Loss: 1.503434 Accuracy: 33.271487
Epoch: 1/4... Step: 600... Training Loss: 1.261374... Validation Loss: 1.458057 Accuracy: 35.876295
Epoch: 1/4... Step: 700... Training Loss: 1.321968... Validation Loss: 1.441256 Accuracy: 36.948834
Epoch: 1/4... Step: 800... Training Loss: 1.356495... Validation Loss: 1.434165 Accuracy: 37.392092
Epoch: 1/4... Step: 900... Training Loss: 1.444345... Validation Loss: 1.423685 Accuracy: 38.389060
Epoch: 1/4... Step: 1000... Training Loss: 1.326540... Validation Loss: 1.407911 Ac

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [1]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [tk for tk in tokens if tk in vocab]
    # Convert words to ids
    tokens = [vocab[tk] for tk in tokens]
        
    # Adding a batch dimension
    #From reviewer suggestion
    text_array = torch.tensor(tokens).view(-1,1)
    # Get the NN output
    hidden = model.init_hidden(1)
    log_probs, _ = model.forward(text_array, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(log_probs)
    
    return pred

In [2]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

NameError: name 'model' is not defined

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [52]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [53]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [54]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [55]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': tensor([[ 0.0803,  0.0615,  0.6567,  0.1393,  0.0623]]),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.